# Evolución del Tour de Francia (1903 - 2025)
**Análisis del cambio de paradigma: De la supervivencia a la optimización fisiológica.**

En este notebook orquestamos el pipeline de datos completo, desde la ingesta de los datos crudos hasta el análisis exploratorio (EDA), utilizando código modular importado de nuestra carpeta `src/`.


Empezamos con las importaciones de lo que vamos a necesitar

In [ ]:
import sys
import os
import pandas as pd

# Obtener la ruta del directorio actual del notebook
current_notebook_dir = os.getcwd()

# La raíz del proyecto está un nivel por encima del directorio 'notebooks'
project_root = os.path.abspath(os.path.join(current_notebook_dir, '..'))

# Añadir la raíz del proyecto al sys.path si no está ya presente
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Ahora se puede importar desde src/
import src.cleaning as cln
import src.features as feat
import src.io as io
import src.viz as viz

# Ignorar warnings de pandas para una presentación limpia
import warnings
warnings.filterwarnings('ignore')


## 1. Pipeline de Procesamiento de Datos
Para mantener este notebook limpio, hemos encapsulado las lógicas de limpieza y cálculo de variables en los módulos `cleaning.py` y `features.py`. A continuación, ejecutamos el pipeline:

In [ ]:
# Empezamos extrayendo y limpiando tdf_stages_master y comparandolo
df_raw_stages = pd.read_csv('../data/raw/tdf_stages_master.csv')
display (df_raw_stages.info())
display (df_raw_stages.head(5))

df_stages_clean = cln.clean_stages(df_raw_stages)
display (df_raw_stages.info())
display (df_raw_stages.head(5))

In [ ]:
# Continuamos con tdf_tours_master
df_raw_tours = pd.read_csv('../data/raw/tdf_tours_master.csv')
display (df_raw_tours.info())
display (df_raw_tours.head(5))

df_tours_clean = cln.clean_tours(df_raw_tours)
display (df_tours_clean.info())
display (df_tours_clean.head(5))

In [ ]:
# Continuamos ahora con tdf_winners_master
df_raw_winners = pd.read_csv('../data/raw/tdf_winners_master.csv')
display (df_raw_winners.info())
display (df_raw_winners.head(5))

df_winners_clean = cln.clean_winners(df_raw_winners)
display (df_winners_clean.info())
display (df_winners_clean.head(5))

In [ ]:
# Finalzamos con tdf_finishers_master
df_raw_finishers = pd.read_csv('../data/raw/tdf_finishers_master.csv')
display (df_raw_finishers.info())
display (df_raw_finishers.head(5))

df_finishers_clean = cln.clean_finishers(df_raw_finishers)
display (df_finishers_clean.info())
display (df_finishers_clean.head(5))

In [ ]:
#Creamos métricas de dureza y fisiología a través de los dataframes ya limpios.
df_stages_feat = feat.calculate_stage_workload(df_stages_clean)
df_analysis = feat.create_hardness_analysis(df_stages_feat, df_tours_clean, df_winners_clean)
df_winners_feat = feat.calculate_physiology(df_winners_clean)

display (df_stages_feat.info())
display (df_stages_feat.head(5))

display (df_analysis.info())
display (df_analysis.head(5))

display (df_winners_feat.info())
display (df_winners_feat.head(5))

## 2. Dimensión Física: Arquitectura y Dureza del Tour
**Pregunta 1:** ¿De qué está hecho el Tour? ¿Siempre ha sido una carrera de alta montaña o esto es un fenómeno moderno?

Para entender cómo ha evolucionado la dureza, no podemos medir solo los kilómetros totales. Hemos creado una métrica propia llamada **Índice de Dureza Ponderada**, que calcula la carga de trabajo de cada etapa multiplicando sus kilómetros por un coeficiente técnico según el terreno:

* **Fórmula:** `Carga de Etapa = Distancia (km) × Peso Técnico`
* **Pesos aplicados:**
  * Montaña (1.0): Máximo desgaste.
  * Media Montaña (0.7): Terreno rompepiernas constante.
  * Contrarreloj Individual (0.6): Esfuerzo aeróbico máximo.
  * Pavé / Contrarreloj Equipos (0.4): Alta tensión y desgaste.
  * Llano (0.2): Menor desgaste al rodar protegidos en el pelotón.

A continuación, agrupamos esta carga de trabajo por décadas para ver la evolución del diseño de la carrera.

In [ ]:
# Gráfico 1: Composición del Esfuerzo por Década
viz.plot_hardness_composition(df_stages_feat)

* **Lectura del Gráfico:** Observamos claramente cómo en las primeras décadas (1900s - 1930s), más del 70% de la dureza provenía de interminables etapas llanas (color azul claro). Sin embargo, a medida que avanzamos hacia la actualidad, la alta montaña (rojo) y la media montaña (naranja) aumentan su importancia. Esto confirma que el Tour moderno está diseñado para castigar mediante el desnivel, no por la acumulación de kilómetros llanos.

### Volumen vs. Densidad
**Pregunta 2:** Si ahora hay más montaña, ¿qué ha pasado con los kilómetros totales? ¿Compensa una cosa a la otra?

Es muy común escuchar que el Tour "antes era mucho más duro" simplemente porque superaba los 5.000 kilómetros totales de recorrido. Para comprobar si esto es cierto o si es un mito, vamos a cruzar dos variables clave: la distancia total de cada edición frente al porcentaje de etapas de montaña (y media montaña) que la componen.

In [ ]:
# Gráfico 2: Relación entre distancia total y porcentaje de montaña
viz.plot_paradigm_shift(df_analysis)

* **Lectura del Gráfico:** Los datos confirman un cambio radical en el modelo de la carrera. La línea de tendencia muestra una correlación inversa de manual. Las ediciones de principios del siglo XX (puntos oscuros, abajo a la derecha) eran auténticos maratones de fondo, larguísimos pero casi planos. Sin embargo, el ciclismo actual (puntos amarillos, arriba a la izquierda) ha recortado unos 1.500 kilómetros del total para concentrar el esfuerzo: menos días de transición y hasta un 40% de etapas con desniveles importantes. Es el paso del ciclista "maratoniano" al escalador explosivo.

### La Paradoja de la Supervivencia
**Pregunta 3:** Si hemos demostrado que el Tour moderno es más intenso y tiene mucha más montaña, ¿significa esto que la carrera actual "destroza" a más ciclistas que en el pasado?

Para analizar esto, vamos a visualizar lo que llamo la "brecha de supervivencia". Representaremos el volumen total de ciclistas que toman la salida frente a los que logran cruzar la meta en París. El área que queda entre ambas métricas representa los abandonos, dándonos una medida muy visual del coste humano de la prueba a lo largo de la historia.

In [ ]:
# Gráfico 3: Evolución de inscritos vs. finalistas (área de abandonos)
viz.plot_survival_gap(df_analysis)

* **Lectura del Gráfico:** El resultado es fascinante porque rompe nuestra hipótesis inicial. Lo lógico sería pensar que a mayor intensidad técnica, más gente se retira. Sin embargo, el área gris (el porcentaje de abandonos) era inmensa en los años 20 y 30, y se ha ido comprimiendo hasta nuestros días, donde la inmensa mayoría del pelotón logra terminar. Esto demuestra que la carrera no se ha hecho más fácil, sino que la profesionalización (medicina, nutrición, entrenamiento por vatios y carreteras asfaltadas) ha elevado la capacidad de supervivencia del ciclista moderno muy por encima de la dureza del trazado.

## 3. Dimensión Fisiológica
### La Edad del Máximo Rendimiento
**Pregunta 4:** ¿Estamos viviendo una era sin precedentes de "niños prodigio" en el ciclismo o siempre ha habido campeones tan jóvenes?

Para analizar cómo ha evolucionado el perfil del campeón, cruzamos la fecha de nacimiento de cada ganador con el año de su victoria (fijando el 15 de julio como fecha central aproximada del Tour). Esto nos da la **edad exacta** del corredor en el momento de alzar el trofeo en París.

Tradicionalmente, la ciencia deportiva establecía que un ciclista de grandes vueltas alcanzaba su "Edad de Oro" (el pico de madurez muscular y resistencia psicológica) entre los 26 y 29 años. Vamos a comprobar si la historia respalda esta teoría.

In [ ]:
# Gráfico 4: Evolución temporal de la edad del ganador
viz.plot_age_evolution(df_winners_feat)

* **Lectura del Gráfico:** Si seguimos la línea de tendencia (en negro) a lo largo de este siglo de historia, el gráfico nos revela cuatro grandes eras fisiológicas:
  1. **Los pioneros (Inicios del s. XX):** En las primeras ediciones, el caos y la supervivencia extrema de la prueba favorecían a ganadores muy jóvenes y temerarios.
  2. **El peso de la experiencia (Años 20 - 30):** A medida que la carrera se estructuraba, la tendencia fue subiendo claramente. El Tour se consolidó como una prueba de desgaste donde la madurez física y la veteranía eran indispensables para aguantar tres semanas.
  3. **La consolidación biológica (Años 40 - 2010):** Vemos cómo la línea se estabiliza y entra justo dentro de la franja gris (26-29 años). Es la época de la profesionalización clásica, coincidiendo exactamente con lo que la ciencia deportiva dictaba como el pico de plenitud muscular y mental.
  4. **Los "Niños Prodigio" (Actualidad):** La curva se desploma en la última década. Los puntos rojos finales rompen todas las reglas históricas. Hoy en día, la recolección masiva de datos, los potenciómetros y la nutrición milimétrica desde categorías junior han permitido que ciclistas de 21 o 22 años ganen el Tour, supliendo los tradicionales "años de experiencia" con pura optimización científica.

### La Evolución del Biotipo"
**Pregunta 5:** Si el Tour actual se decide en la alta montaña y los ciclistas son cada vez más jóvenes, ¿ha cambiado también su biotipo físico?

Para responder a esto, analizamos el Índice de Masa Corporal (BMI = Peso / Altura²) de los ganadores. En el ciclismo moderno, la métrica reina es el "w/kg" (vatios por kilo de peso); para subir más rápido, necesitas generar mucha potencia, pero pesando lo menos posible.

*Nota de Calidad de Datos:* Hemos filtrado el análisis desde 1950 en adelante. Los registros antropométricos de principios de siglo (antes de la Segunda Guerra Mundial) a menudo se basaban en estimaciones periodísticas y carecen del rigor médico necesario para un análisis estadístico serio.

In [ ]:
# Gráfico 5: Evolución del Índice de Masa Corporal (BMI)
viz.plot_bmi_evolution(df_winners_feat)

* **Lectura del Gráfico:** La curva de regresión polinómica (línea punteada) dibuja una caída constante y espectacular. En los años 50 y 60, los ganadores solían tener un biotipo mucho más robusto (rondando un BMI de 23 o 24), ideal para rodar con potencia en el llano. Sin embargo, a medida que la carrera incluyó más desniveles, el cuerpo del ganador se fue "afilando". Hoy en día, todos los datos convergen en la franja verde inferior (BMI entre 18.5 y 20.5), rozando el límite de la delgadez extrema. El Tour ya no lo ganan corredores corpulentos, sino "pesos pluma" hiper-especializados.

## 4. Dimensión Geopolítica: La Globalización del Pelotón
**Pregunta 6:** ¿Ha dejado el Tour de ser una carrera de dominio exclusivamente europeo para convertirse en un evento verdaderamente global?

Para finalizar nuestro análisis, exploramos la procedencia de la "clase trabajadora" del Tour: los finalistas. Utilizando el dataset completo de ciclistas que lograron cruzar la meta en París (`tdf_finishers`), hemos analizado la nacionalidad de miles de corredores a lo largo de más de un siglo. 

Hemos agrupado a las cinco grandes potencias históricas del ciclismo europeo (Francia, Bélgica, Italia, España y Países Bajos) y las hemos comparado contra la proporción de ciclistas provenientes del "Resto del Mundo".

In [ ]:
# Gráfico 6: Evolución geopolítica de los finalistas (Área apilada al 100%)
viz.plot_globalization(df_finishers_clean)

* **Lectura del Gráfico:** El gráfico de área apilada revela una transformación demográfica total. Hasta la década de 1970, el Tour era un feudo cerrado: casi el 90% de los ciclistas que lograban terminar la carrera provenían de las cinco potencias clásicas (dominadas abrumadoramente por la franja azul francesa y la negra belga). Sin embargo, a partir de los años 80 y 90, coincidiendo con la profesionalización y la televisión global, la franja gris ("Resto del Mundo") comienza a devorar el gráfico. Hoy en día, el pelotón es un crisol global donde las naciones históricas apenas representan a la mitad de los supervivientes.

## 5. Conclusiones Generales

Tras analizar más de un siglo de datos históricos del Tour de Francia cruzando las métricas de recorrido, supervivencia, demografía y fisiología, llegamos a tres conclusiones fundamentales respaldadas por los datos:

1. **El cambio de paradigma en la Dureza:** El mito de que el Tour "antes era más duro" es engañoso. Los datos demuestran una correlación inversa histórica: la organización ha recortado drásticamente la distancia total (de más de 5.000 km a unos 3.400 km) para poder concentrar el esfuerzo. Hemos pasado de maratones llanos de supervivencia a una carrera de alta intensidad donde la montaña y media montaña representan ya el 40% del esfuerzo total.

2. **La "Paradoja de la Supervivencia":** A pesar de que la densidad técnica y los desniveles son más extremos que nunca, la brecha histórica de abandonos casi ha desaparecido. Esto demuestra que la ciencia deportiva, la nutrición, la biomecánica y el asfalto moderno han evolucionado a un ritmo superior al de la propia dureza de la carrera. Hoy en día, la preparación es tan perfecta que la inmensa mayoría del pelotón sobrevive a las tres semanas.

3. **Adaptación extrema del Factor Humano (Fisiología y Sociología):** La máquina humana se ha moldeado para sobrevivir a este nuevo paradigma de alta montaña. El biotipo del ganador corpulento ha desaparecido a favor de "pesos pluma" hiper-especializados (BMI inferior a 21) y la tecnología ha suplido a la "experiencia", abriendo la era de los niños prodigio ganando a los 21 años. Además, sociológicamente, el Tour ha dejado de ser un feudo puramente europeo; el pelotón se ha globalizado por completo, y las grandes potencias históricas (Francia, Bélgica, Italia, España y Países Bajos) ya conforman menos de la mitad del pelotón superviviente.